# Project 4 - Train a Smartcab to Drive

## Implement a Basic Driving Agent

To begin, your only task is to get the smartcab to move around in the environment. At this point, you will not be concerned with any sort of optimal driving policy.

To complete this task, simply have your driving agent choose a random action from the set of possible actions (None, `'forward'`, `'left'`, `'right'`) at each intersection. Set the simulation deadline enforcement, **`enforce_deadline`** to **`False`** and observe how it performs.

### Code Snippet:
    def random_action(self):
        from random import choice
        return choice([None, 'forward', 'left', 'right'])

### Question 1:

**Observe what you see with the agent's behavior as it takes random actions. Does the smartcab eventually make it to the destination? Are there any other interesting observations to note?**

1. In almost 65% of the trials, the smartcab eventually makes it to the destination within the hard time limit (-100).
2. Other observations that were not mentioned in the problem statement:
    1. The agent seems to never reach the destination within the time limit (it always goes to negative time)
    2. Environment wrap around itself (East-most connected to West-most, and North-most connected to South-most.)
    3. Disobeying traffic lows results in -1 rewards, and the action is not performed
    4. Reaching the goal results in +10 rewards
    5. Performing a legal move results in either +2 or -0.5 (by looking at the code, I figured it is based on whether the smartcab followed the `next_waypoint` suggestion by the planner or not)
    
    

## Inform the Driving Agent

Now that your driving agent is capable of moving around in the environment, your next task is to identify a set of states that are appropriate for modeling the smartcab and environment.

The main source of state variables are the current inputs at the intersection, but not all may require representation. You may choose to explicitly define states, or use some combination of inputs as an implicit state.

At each time step, process the inputs and update the agent's current state using the `self.state` variable. Continue with the simulation deadline enforcement `enforce_deadline` being set to `False`, and observe how your driving agent now reports the change in state as the simulation progresses.

### Code Snippet
    # TODO: Update state
    legal_right = inputs['light'] == 'green' or inputs['left'] != 'forward'
    legal_forward = inputs['light'] == 'green'
    legal_left = inputs['light'] == 'green' and inputs['oncoming'] != 'right' and inputs['oncoming'] != 'forward'
    self.state = {'next_waypoint': self.next_waypoint, 
                  'legal_right': legal_right, 
                  'legal_forward': legal_forward, 
                  'legal_left': legal_left}

### Question 2:

**What states have you identified that are appropriate for modeling the smartcab and environment? Why do you believe each of these states to be appropriate for this problem?**

1. I was able to identify four states that would model the smarcab and the environment:
    1. `next_waypoint`: To represent the direction suggested by the planner
    2. `legal_right`: To represent whether it is legal to turn right at the intersection
    3. `legal_forward`: To represent whether it is legal to keep moving forward at the intersection
    4. `legal_left`: To preresent whether it is legal to turn left at the intersection
    
2. Appropriateness of these states were achieved by:
    1. `next_waypoint`: Since the `next_waypoint` affects the reward that would be achieved at each state, it is important to have it as part of our state as a representation of the smartcab planner
    2. `legal_right`, `legal_forward`, and `legal_left`: These three variables embed the information received by the sensors of the smartcab. Instead of using sensor data directly (`light`, `right`, `oncoming`, and `left`), these three states allow us to achieve:
        1. Reduce the state-space representing the enironment from $2 x 3 x 3 x 3 = 54$ (`Light = ['green', 'red']`, `right = oncoming = left = ['right', 'forward', 'left']`. Note that dummy agents don't have `'None'` as an option) to $2 x 2 x 2 = 8$ (`legal_right = legal_forward = legal_left = [True, False]`)
        2. While reducing the state-space, the information needed for the smartcab to make its decision is still preserved. The value of interest behind knowing the sensor data for an intersection is to know whether a right, left, or forward move is legal. These three states keep that information. In fact, they represent this information.
        

### Question 3:

**How many states in total exist for the smartcab in this environment? Does this number seem reasonable given that the goal of Q-Learning is to learn and make informed decisions about each state? Why or why not?**

1. Number of states that exist are $2 x 2 x 2 x 3 = 24$ (`legal_right`, `legal_forward`, and `legal_left` have two possible values, `True` and `False`. `next_waypoint` has three possible values, `right`, `forward`, and `left`. Note that the planner doesn't give `None` as an option).

2. The number of states seems reasonable for a Q-Learning since it is low enough. These 24 states could easily be traversed several times by running the simulation several times. 

## Implement a Q-Learning Driving Agent

### Question 4:

**What changes do you notice in the agent's behavior when compared to the basic driving agent when random actions were always taken? Why is this behavior occurring?**

## Improve the Q-Learning Driving Agent

### Question 5:

**Report the different values for the parameters tuned in your basic implementation of Q-Learning. For which set of parameters does the agent perform best? How well does the final driving agent perform?**

### Question 6:

**Does your agent get close to finding an optimal policy, i.e. reach the destination in the minimum possible time, and not incur any penalties? How would you describe an optimal policy for this problem?**